# Essay Keystrokes Project

This project is based on the Kaggle competition [Linking Writing Processes to Writing Quality](https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality). The train data set for the competition includes scores for about 2.7k essays as well as a set of logs with thousands of records for each essay with over 8 million records total.  
  
The goal of the project is to use the data about each student's writing process to predict the score that they receive on their essay while minimizing rmse.  My project includes two notebooks: one for preprocessing and one for modeling.

## Project Summary

<img src="Essay_DFD.png" alt=" " width="900" height="900" style="float:left; margin-right:10px;">

## Modeling Notebook

In this notebook, I apply the preprocessing function from my previous notebook to prepare the data for modeling. Then, I build regression models to predict the score for each essay and use an iterative approach to build a model to minimize rmse.

Because the test data is hosted privately on Kaggle, I use cross validation scores on the train data to evaluate each model's performance in this notebook. My final model performs with a .626 rmse on the unseen test data on Kaggle.

### 1. Imports and Applying Preprocessing Function

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import pandas as pd

import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor

from xgboost import XGBRegressor

In [2]:
train_df = pd.read_csv('Data/train_logs.csv')
train_scores_df = pd.read_csv('Data/train_scores.csv')

In [3]:
def preprocess(log_df):
    
    # Action Time
    action_time = log_df.groupby(by='id')['action_time'].sum()
    action_time_df = pd.DataFrame({'id':action_time.index, 'action_time':action_time.values})
    action_time_df['action_time'] = action_time_df['action_time']
    action_time_df['action_time_%_total'] = action_time_df['action_time'] / 1800000
    
    # Final Word Count
    final_word_count = log_df.groupby('id')['word_count'].last()
    word_count_df = pd.DataFrame({'id':final_word_count.index,'word_count':final_word_count.values})
    final_record_id_series = log_df.groupby('id')['event_id'].last()
    final_record_ids = final_record_id_series.to_dict()
    
    def total_words(group):
        final_record_id = final_record_ids[group.name]
        word_counts = group[group['event_id'] <= final_record_id]['word_count']
        word_diffs = word_counts.diff().where(lambda x: x > 0).sum()  
    
        return word_diffs
    
    total_words = log_df.groupby('id').apply(total_words)
    total_words_df = pd.DataFrame({'id':total_words.index,'total_words':total_words.values})
    word_count_df = pd.merge(word_count_df, total_words_df, on='id')
    
    # Final Event Count
    event_count = log_df.groupby('id')['event_id'].last()
    event_count_df = pd.DataFrame({'id':event_count.index,'event_count':event_count.values})
    
    # Activity
    activities = ['Input', 'Move', 'Nonproduction', 'Paste', 'Remove/Cut', 'Replace']
    
    log_df.loc[log_df['activity'].str.contains('From',case=False), 'activity'] = 'Move'
    log_df.loc[log_df['activity'].str.contains('From',case=False), 'activity'] = 'Move'
    activity_data = log_df.groupby(by='id')['activity'].value_counts().unstack(fill_value=0)
    
    for activity in activities:
        if activity in activity_data.columns:
            activity_data[activity] = activity_data[activity]
        else:
            activity_data[activity] = 0
    
    activity_df = activity_data
    activity_df['id'] = activity_df.index
    activity_df.reset_index(drop=True,inplace=True)
    
    pivot = log_df.pivot_table(values='action_time', index='id', columns='activity', aggfunc=['sum']).fillna(0)
    pivot.columns = pivot.columns.droplevel(level=0)
    
    for activity in activities:
        if activity in pivot.columns:
            pivot[activity] = pivot[activity]
        else:
            pivot[activity] = 0
    
    new_columns = [col + '_time' for col in pivot.columns]
    pivot.columns = new_columns
    pivot['id'] = pivot.index
    pivot.reset_index(drop=True,inplace=True)
    
    # Time of First and Last Input
    first_input = log_df[log_df['activity'] == 'Input'].groupby('id').first()
    first_input_df = pd.DataFrame({'id':first_input.index,'first_input':first_input['down_time']})
    first_input_df.reset_index(drop=True,inplace=True)
    last_input = log_df[log_df['activity'] == 'Input'].groupby('id').last()
    last_input_df = pd.DataFrame({'id':last_input.index,'last_input':last_input['up_time']})
    last_input_df.reset_index(drop=True,inplace=True)
    first_last_input_df = pd.merge(first_input_df,last_input_df,on='id')
    
    # Inputs
    columns = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',',
               'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified', 'Control',
               '"', '-', '?', ';', '=', 'Tab', '/', 'Rightclick', ':', '(', ')', '\\', 'ContextMenu',
               'End', '!', 'Meta', 'Alt', '[', 'c', 'v', 'NumLock', 'Insert', 'Home', 'z', 'AudioVolumeDown',
               'F2', 'a', 'x', 'AudioVolumeUp', '$', '>', ']', '*', '%', '&', 'Dead', 's', 'Escape',
               'ModeChange', 'F3', '<', 'AudioVolumeMute', 'F15', '+', 'ScrollLock', 'Process', 'PageDown',
               't', 'i', '_', '`', 'PageUp', '{', '0', '#', 'Middleclick', '1', '5', 'F12', '\x97',
               'OS', 'e', '@', 'F11', 'r', 'MediaTrackNext', 'y', 'm', 'n', 'b', 'Clear', 'MediaPlayPause', 'o']
    
    down_event_data = log_df.groupby(by='id')['down_event'].value_counts().unstack(fill_value=0)
    
    for col in columns:
        if col in down_event_data.columns:
            down_event_data[col] = down_event_data[col]
        else:
            down_event_data[col] = 0
    
    down_event_df = down_event_data[columns]
    down_event_df.reset_index(inplace=True)
    
            
    # Pauses
    def add_previous_down_time(group):
        group['previous_down_time'] = group['down_time'].shift(1).fillna(group['down_time'])
        return group[['id','down_time','previous_down_time']]

    pause_df = log_df.groupby('id').apply(add_previous_down_time)
    
    pause_df['down_time_difference'] = pause_df['down_time'] - pause_df['previous_down_time']
    pause_df['pauses'] = 0
    pause_df.loc[pause_df['down_time_difference']>=5000,'pauses'] = 1
    
    pause_df['pause_time'] = 0
    pause_df.loc[pause_df['down_time_difference']>=5000,'pause_time'] = pause_df['down_time_difference']
    
    pause_df = pause_df.reset_index(drop=True)
    pause_df = pause_df[['id','pauses','pause_time']]
    pause_df = pause_df.groupby('id').sum()
    
    # Merge DataFrames
    full_df = pd.merge(action_time_df, word_count_df, on = 'id')
    full_df = pd.merge(full_df, pivot, on = 'id')
    full_df = pd.merge(full_df, event_count_df, on = 'id')
    full_df = pd.merge(full_df, activity_df, on = 'id')
    full_df = pd.merge(full_df, first_last_input_df, on = 'id')
    full_df = pd.merge(full_df, down_event_df, on = 'id')
    full_df = pd.merge(full_df, pause_df, on = 'id')
    
    # Sentence Length
    full_df['end_punctuation'] = full_df['.'] + full_df['!'] + full_df['?']
    full_df['sentence_length'] = full_df['word_count'] / full_df['end_punctuation']
    full_df.loc[full_df['end_punctuation']==0,'sentence_length'] = full_df['word_count']
    
    # Ratios
    full_df['Move_Ratio'] = full_df['Move'] / full_df['event_count']
    full_df['Paste_Ratio'] = full_df['Paste'] / full_df['event_count']
    full_df['Replace_Ratio'] = full_df['Replace'] / full_df['event_count']
    full_df['Nonproduction_Ratio'] = full_df['Nonproduction'] / full_df['event_count']
    full_df['Remove_Ratio'] = full_df['Remove/Cut'] / full_df['event_count']
    full_df['Input_Ratio'] = full_df['Input'] / full_df['event_count']
    full_df['revision_events'] = full_df['Move'] + full_df['Paste'] + full_df['Replace'] + full_df['Remove/Cut']
    full_df['Revision_Ratio'] =  full_df['revision_events'] / full_df['event_count']
    full_df['characters_per_pause'] = full_df['q'] / full_df['pauses']
    full_df.loc[full_df['pauses']==0,'characters_per_pause'] = full_df['q']
    full_df['Move_Time_Ratio'] = full_df['Move_time'] / full_df['action_time']
    full_df['Paste_Time_Ratio'] = full_df['Paste_time'] / full_df['action_time']
    full_df['Replace_Time_Ratio'] = full_df['Replace_time'] / full_df['action_time']
    full_df['Nonproduction_Time_Ratio'] = full_df['Nonproduction_time'] / full_df['action_time']
    full_df['Remove_Time_Ratio'] = full_df['Remove/Cut_time'] / full_df['action_time']
    full_df['Input_Time_Ratio'] = full_df['Input_time'] / full_df['action_time']
    full_df['revision_time'] = full_df['Move_time'] + full_df['Paste_time'] + full_df['Replace_time'] + full_df['Remove/Cut_time']
    full_df['Revision_Time_Ratio'] =  full_df['revision_time'] / full_df['action_time']
    full_df['action_time_per_pause'] = full_df['action_time'] / full_df['pauses']
    full_df.loc[full_df['pauses']==0,'action_time_per_pause'] = full_df['action_time']
    full_df['action_time_per_pause_time'] = full_df['action_time'] / full_df['pause_time']
    full_df.loc[full_df['pause_time']==0,'action_time_per_pause_time'] = full_df['action_time']
    full_df['characters_per_word'] = full_df['q'] / full_df['total_words']
    full_df.loc[full_df['total_words']==0,'characters_per_word'] = full_df['q']
    
    return full_df

In [4]:
## Applying preprocess function

full_df = preprocess(train_df)

In [5]:
## Merging with scores

full_df = pd.merge(full_df,train_scores_df,on='id')
full_df

,id,action_time,action_time_%_total,word_count,total_words,Input_time,Move_time,Nonproduction_time,Paste_time,Remove/Cut_time,...,Replace_Time_Ratio,Nonproduction_Time_Ratio,Remove_Time_Ratio,Input_Time_Ratio,revision_time,Revision_Time_Ratio,action_time_per_pause,action_time_per_pause_time,characters_per_word,score
0,001519c8,297243,0.165135,255,348.0,243731.0,0.0,18506.0,0.0,34130.0,...,0.002947,0.062259,0.114822,0.819972,35006.0,0.117769,4644.421875,0.300140,4.652299,3.5
1,0022f953,275391,0.152995,320,369.0,237891.0,0.0,13781.0,71.0,23550.0,...,0.000356,0.050042,0.085515,0.863830,23719.0,0.086128,5859.382979,0.255246,4.037940,3.5
2,0042269b,421201,0.234001,404,549.0,353718.0,0.0,33951.0,0.0,32905.0,...,0.001489,0.080605,0.078122,0.839784,33532.0,0.079610,12388.264706,0.427964,5.289617,6.0
3,0059420b,189596,0.105331,206,244.0,167790.0,0.0,3062.0,160.0,18410.0,...,0.000918,0.016150,0.097101,0.884987,18744.0,0.098863,6319.866667,0.315773,4.254098,2.0
4,0075873a,313702,0.174279,252,339.0,266515.0,0.0,6988.0,0.0,40199.0,...,0.000000,0.022276,0.128144,0.849580,40199.0,0.128144,6402.081633,0.340198,4.545723,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,499670,0.277594,273,620.0,426990.0,0.0,5203.0,0.0,67253.0,...,0.000448,0.010413,0.134595,0.854544,67477.0,0.135043,20819.583333,0.542786,4.587097,3.5
2467,ffbef7e5,214221,0.119012,438,450.0,203403.0,0.0,6583.0,0.0,4118.0,...,0.000546,0.030730,0.019223,0.949501,4235.0,0.019769,5100.500000,0.260813,4.164444,4.0
2468,ffccd6fd,231580,0.128656,201,213.0,214677.0,0.0,10232.0,0.0,6671.0,...,0.000000,0.044183,0.028806,0.927010,6671.0,0.028806,6811.176471,0.236213,4.549296,1.5
2469,ffec5b38,289439,0.160799,413,472.0,263216.0,0.0,5624.0,0.0,20599.0,...,0.000000,0.019431,0.071169,0.909401,20599.0,0.071169,9647.966667,0.453550,5.002119,5.0


In [6]:
## Checking for nulls

full_df.isnull().sum().sum()

0

## 2. Preparing for Modeling

## 2. Modeling Round 1

In this section, I use an iterative modeling approach to predict the scores of the final essays while minimizing rmse. Due to the low volume of records for modeling, I use cross validation scores to evaluate my models rather than splitting into train and test. I use the competition test data as the final way to evaluate my model's performance.

### 2a. Defining X and Y and Pipeline Setup

In [7]:
# Defining independent and dependent variables

X = full_df.drop(['id','score'],axis=1)
y = full_df['score']

In [8]:
# Creating column transformer

ct = ColumnTransformer(transformers = [
    ('ss', StandardScaler(), X.columns)
])

### 2b. Dummy: Baseline Model

I start by building a dummy model to create baseline rates to evaluate the performance of the other models I create. I evaluate all models in terms of rmse and r2 even though rmse is the only metric used for the competition. Using r2 as well provides a more complete description of how each model performs.

In [9]:
# Creating dummy pipeline

dum_pipe = Pipeline([
    ('ct', ct),
    ('dummy', DummyRegressor(strategy='mean'))
])

In [10]:
dum_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('dummy', DummyRegressor())])

In [11]:
# Calculating dummy rmse

dummy_rmse = round(np.sqrt(mean_squared_error(y,dum_pipe.predict(X))),4)
dummy_rmse

1.0247

In [12]:
# Calculating dummy r2

dummy_r2 = round(r2_score(y,dum_pipe.predict(X)),4)
dummy_r2

0.0

In [13]:
# Creating results dataframe and appending dummy scores

results_df = pd.DataFrame({'Model':'Dummy','RMSE':dummy_rmse,'R2':dummy_r2}, index=[0])
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0


The dummy regressor, which makes its predictions by predicting the mean score from the train data every single time, tends to be off by a little over 1 on an average prediction, and it accounts for none of the variation in the data. The R2 makes sense because there is no variation in the predictions; using this rmse as a baseline will show the value the models provide above simply predicting every writer achieves an average score.

### 2c. First Simple Model: Random Forest

Because the data does not meet the assumptions of linear regression, I focus on using ensemble methods that are less sensitive to issues such as multicollinearity. The first model I use is a random forest regressor. I create a function to add the results for each model to a rolling dataframe to make it easier to compare model effectiveness.

In [14]:
# Creating random forest pipeline

rf_pipe = Pipeline([
    ('ct', ct),
    ('RF', RandomForestRegressor(max_depth=5,random_state=42))
])

In [15]:
# Creating a function to append the results for each model to a rolling dataframe

def results(pipe, results_df, X, y):
    
    # Defining predictions
    y_pred = pipe.predict(X)
    
    # Define a function to calculate RMSE
    def rmse(y, y_pred):
        return -np.sqrt(((y - y_pred) ** 2).mean())

    # Make RMSE a scorer using make_scorer
    rmse_scorer = make_scorer(rmse, greater_is_better=False)
    cv_rmse = round(cross_val_score(pipe, X, y, cv=5, scoring=rmse_scorer).mean(),4)
    
    # Make r2_scorer
    r2_scorer = make_scorer(r2_score)
    cv_r2 = round(cross_val_score(pipe, X, y, cv=5, scoring=r2_scorer).mean(),4)
    
    # Extracting model name from the last step in the pipeline
    model_name = pipe.steps[-1][0]
    
    # Appending results to the DataFrame
    results_df = results_df.append({'Model': model_name, 'RMSE': cv_rmse, 'R2': cv_r2}, ignore_index=True)
    
    return results_df

In [16]:
rf_pipe.fit(X,y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('RF', RandomForestRegressor(max_depth=5, random_state=42))])

In [17]:
# Appending results to the rolling dataframe

results_df = results(rf_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801


The random forest model performs substantially better than the dummy model, explaining 58% of the variance in the data and achieving an rmse of .66. Next, I use a gridsearch to tune hyperparameters and try to maximize model performance.

In [18]:
# Selecting parameters for gridsearch

params_rf = {
    'RF__n_estimators' : [50, 100, 200],
    'RF__max_depth' : [3, 5, 7],
    'RF__min_samples_split': [2, 5, 7],
    'RF__min_samples_leaf': [1, 2, 4]
    
}

In [19]:
# Instantiating gridsearch variable

gs_rf = GridSearchCV(
    estimator = rf_pipe,
    param_grid = params_rf,
    cv = 5,
    verbose = 1)

In [20]:
# For time purposes, I comment these lines out and use the parameters in the code that follows.

# gs_rf.fit(X,y)

In [21]:
# gs_rf.best_params_

In [22]:
# Creating a pipeline with the gridsearch parameters

gs_rf_pipe = Pipeline([
    ('ct', ct),
    ('GS_RF',RandomForestRegressor(max_depth=7,
                                   n_estimators=200,
                                   min_samples_leaf=2,
                                   min_samples_split=2,
                                   random_state=42))
])

In [23]:
gs_rf_pipe.fit(X,y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('GS_RF',
                 RandomForestRegressor(max_depth=7, min_samples_leaf=2,
                                       n_estimators=200, random_state=42))])

In [24]:
# Appending results to the rolling dataframe

results_df = results(gs_rf_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889


Using the grid search hyperparameters marginally improves the results.

### 2d. Gradient Boost

Next, I try a gradient boosting regressor to see if it can outperform the random forest.

In [25]:
# Creating the pipeline

gb_pipe = Pipeline([
    ('ct', ct),
    ('GB', GradientBoostingRegressor(max_depth=3,random_state=42))
])

In [26]:
gb_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('GB', GradientBoostingRegressor(random_state=42))])

In [27]:
# Appending the results to the rolling dataframe

results_df = results(gb_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010


The gradient boosting regressor performs slightly better than the random forest model did both in terms of rmse and r2.  I use a grid search to tune hyper parameters to see if that can further boost performance.

In [28]:
# Defining parameters

params_gb = {
    'GB__learning_rate' : [.01, .05, .1, .2, .3],
    'GB__n_estimators': [50, 100, 200],
    'GB__max_depth': [3, 5, 7]
}

In [29]:
# Instantiating grid search

gs_gb = GridSearchCV(
    estimator = gb_pipe,
    param_grid = params_gb,
    cv = 5,
    verbose = 1
)

In [30]:
# Commented out to save time/computing power

# gs_gb.fit(X,y)

In [31]:
# gs_gb.best_params_

In [32]:
# Creating new gradient boost pipeline with hyperparameters from grid search

gs_gb_pipe = Pipeline([
    ('ct', ct),
    ('GS_GB', GradientBoostingRegressor(learning_rate=.05,
                                        max_depth=3,
                                        n_estimators=100,
                                        random_state=42))
])

In [33]:
gs_gb_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('GS_GB',
                 GradientBoostingRegressor(learning_rate=0.05,
                                           random_state=42))])

In [34]:
# Appending results to the rolling dataframe

results_df = results(gs_gb_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055


Once again, the model performs marginally better after tuning hyper parameters using the grid search results.

### 2e. ADA Boost

For my third model, I use an ADA Boost regressor.

In [35]:
# Creating the pipeline

ada_pipe = Pipeline([
    ('ct', ct),
    ('ADA', AdaBoostRegressor(random_state=42))
])

In [36]:
ada_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('ADA', AdaBoostRegressor(random_state=42))])

In [37]:
# Appending results to the dataframe

results_df = results(ada_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622


The model starts out performing worse than the previous two. I use grid search to tune hyperparameters to see if the model can improve substantially.

In [38]:
params_ada = {
    'ADA__learning_rate' : [.01, .05, .1, .2, .3],
    'ADA__n_estimators' : [50, 100, 200],
    'ADA__loss' : ['linear', 'square', 'exponential']
}

In [39]:
# Instantiating grid search

gs_ada = GridSearchCV(
    estimator = ada_pipe,
    param_grid = params_ada,
    cv = 5,
    verbose = 1
)

In [40]:
# gs_ada.fit(X,y)

In [41]:
# gs_ada.best_params_

In [42]:
# Creating the pipeline with tuning

gs_ada_pipe = Pipeline([
    ('ct',ct),
    ('GS_ADA',AdaBoostRegressor(learning_rate=.2,
                                n_estimators=50,
                                loss='exponential',
                                random_state=42))
])

In [43]:
gs_ada_pipe.fit(X,y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'revision_time',
       'Revision_Time_Ratio', 'action_time_per_pause',
       'action_time_per_pause_time', 'characters_per_word'],
      dtype='object', length=133))])),
                ('GS_ADA',
                 AdaBoostRegressor(learning_rate=0.2, loss='exponential',
                                   random_state=42))])

In [44]:
# Appending results to the rolling data frame

results_df = results(gs_ada_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622
6,GS_ADA,0.6751,0.5654


The model performs better after hyperparameter tuning, but it is still not as good as the gradient boosting regressor or the random forest regressor.

### 2h. XGBoost

The last model I try is XGB.

In [45]:
# Creating the pipeline

xgb_pipe = Pipeline([
    ('ct', ct),
    ('XGB', XGBRegressor(max_depth=3,random_state=42))
])

In [46]:
xgb_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'r...
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=3, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1, random_state=42,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

In [47]:
# Appending results to the rolling dataframe

results_df = results(xgb_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622
6,GS_ADA,0.6751,0.5654
7,XGB,0.6732,0.5674


XGB performs similarly to the adaptive boosting regressor before hyperparameter tuning.

In [48]:
params_xgb = {
    'XGB__learning_rate' : [.01, .05, .1, .2, .3],
    'XGB__n_estimators': [50, 100, 200],
    'XGB__max_depth': [3, 5, 7],
    'XGB__min_child_weight': [1, 3, 5]
}

In [49]:
# Instantiating first grid search

gs_xgb = GridSearchCV(
    estimator = xgb_pipe,
    param_grid = params_xgb,
    cv = 5,
    verbose = 1
)

In [50]:
# gs_xgb.fit(X, y)

In [51]:
# gs_xgb.best_params_

In [52]:
# Creating pipeline

gs_xgb_pipe = Pipeline([
    ('ct',ct),
    ('GS_XGB',XGBRegressor(learning_rate = .05, 
                           max_depth = 3,
                           min_child_weight = 3,
                           n_estimators = 100,
                           random_state=42))
])

In [53]:
gs_xgb_pipe.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Move_time', 'Nonproduction_time', 'Paste_time',
       'Remove/Cut_time', 'Replace_time',
       ...
       'Paste_Time_Ratio', 'Replace_Time_Ratio', 'Nonproduction_Time_Ratio',
       'Remove_Time_Ratio', 'Input_Time_Ratio', 'r...
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.05,
                              max_delta_step=0, max_depth=3, min_child_weight=3,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              random_state=42, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [54]:
results_df = results(gs_xgb_pipe, results_df, X, y)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622
6,GS_ADA,0.6751,0.5654
7,XGB,0.6732,0.5674
8,GS_XGB,0.6414,0.6074


After hyperparameter tuning, the xgb model is my strongest model so far, performing slightly better than the gradient boosting regressor.  Overall, the random forest model, gradient boosting model and xgb model are my three strongest. I use them to perform one more round of feature selection and feature engineering to try to eliminate as many unnecessary features as possible and to create new features from the most important ones.

## 3. Feature Selection, Feature Engineering, and Modeling Preparation

In the code that follows, I review the feature importances of my three strongest models and create a second preprocessing function to perform feature selection and feature engineering based on what is learned.

### 3a. Feature Selection

In [55]:
# Finding the feature importances from the random forest pipeline

rf_mod = gs_rf_pipe.named_steps['GS_RF']
rf_impt = rf_mod.feature_importances_

In [56]:
# Creating a dataframe with feature names and importances

cols = list(X.columns)
impt_df = pd.DataFrame({'features': cols, 'rf_impt':rf_impt})

In [57]:
# Repeating the process for gradient boost pipeline

gb_mod = gs_gb_pipe.named_steps['GS_GB']
gb_impt = gb_mod.feature_importances_
impt_df['gb_impt'] = gb_impt

In [58]:
# Repeating the process for xgb pipeline

xgb_mod = gs_xgb_pipe.named_steps['GS_XGB']
xgb_impt = xgb_mod.feature_importances_
impt_df['xgb_impt'] = xgb_impt

In [59]:
# Creating features for total and mean importance

impt_df['sum'] = impt_df['rf_impt'] + impt_df['gb_impt'] + impt_df['xgb_impt']
impt_df['avg'] = impt_df['sum'] / 3

In [60]:
# Reviewing features with no importance in any model

impt_df.loc[impt_df['sum']==0]

,features,rf_impt,gb_impt,xgb_impt,sum,avg
5,Move_time,0.0,0.0,0.0,0.0,0.0
12,Move,0.0,0.0,0.0,0.0,0.0
48,ContextMenu,0.0,0.0,0.0,0.0,0.0
61,F2,0.0,0.0,0.0,0.0,0.0
64,AudioVolumeUp,0.0,0.0,0.0,0.0,0.0
68,*,0.0,0.0,0.0,0.0,0.0
71,Dead,0.0,0.0,0.0,0.0,0.0
72,s,0.0,0.0,0.0,0.0,0.0
73,Escape,0.0,0.0,0.0,0.0,0.0
74,ModeChange,0.0,0.0,0.0,0.0,0.0


Several features have no importance to any of the model performances, so I drop them from the dataframe to reduce the noisiness of the data.

In [61]:
impt_df = impt_df.loc[impt_df['sum']!=0]
impt_df

,features,rf_impt,gb_impt,xgb_impt,sum,avg
0,action_time,0.002164,0.000446,0.007654,0.010264,0.003421
1,action_time_%_total,0.002206,0.002323,0.000000,0.004529,0.001510
2,word_count,0.489217,0.480467,0.257014,1.226699,0.408900
3,total_words,0.008508,0.002416,0.006210,0.017135,0.005712
4,Input_time,0.005354,0.003044,0.008104,0.016502,0.005501
...,...,...,...,...,...,...
128,revision_time,0.002617,0.001776,0.005844,0.010236,0.003412
129,Revision_Time_Ratio,0.002730,0.000991,0.000000,0.003722,0.001241
130,action_time_per_pause,0.004084,0.000315,0.007210,0.011610,0.003870
131,action_time_per_pause_time,0.003558,0.000410,0.005743,0.009711,0.003237


Next, I review the feature with the strongest average importances on the models.

In [62]:
impt_df.sort_values(by='avg',ascending=False).head(10)

,features,rf_impt,gb_impt,xgb_impt,sum,avg
2,word_count,0.489217,0.480467,0.257014,1.226699,0.408900
19,q,0.163411,0.193129,0.171903,0.528443,0.176148
27,",",0.071597,0.102099,0.047402,0.221097,0.073699
11,Input,0.032447,0.026996,0.064293,0.123736,0.041245
132,characters_per_word,0.030733,0.032870,0.012828,0.076431,0.025477
112,sentence_length,0.015624,0.022673,0.014053,0.052350,0.017450
22,Shift,0.014236,0.017756,0.019903,0.051895,0.017298
37,-,0.004788,0.016971,0.023785,0.045544,0.015181
30,Enter,0.009766,0.017266,0.011613,0.038646,0.012882
10,event_count,0.007639,0.006578,0.022680,0.036897,0.012299


Interestingly, word count is by far the most important feature with an importance more than double that of the next most important feature and more than four times as important as the third most important feature. The number of characters also has a strong importance. The high importance of these two features suggest that more is more: essays with more words and more characters tend to earn higher scores.

The importance of features like commas, characters per word, and sentence length point towards sentence complexity as also correlating with final score.

### 3b. Feature Engineering and Data Processing Function

Given what has been learned from reviewing the feature importances, I create a second preprocessing function to remove features that have no importance to the models and to engineer some new features.

In [63]:
def preprocess2(df):
    
    keep_cols = ['id','score','action_time','action_time_%_total','word_count','total_words','Input_time','Nonproduction_time',
                 'Paste_time','Remove/Cut_time','Replace_time','event_count','Input','Move','Nonproduction',
                 'Paste','Remove/Cut','Replace','first_input','last_input','q','Space','Backspace','Shift',
                 'ArrowRight','Leftclick','ArrowLeft','.',',','ArrowDown','ArrowUp','Enter','CapsLock',"'",
                 'Delete','Unidentified','Control','"','-','?',';','=','Tab','/','Rightclick',':','(',')',
                 '\\','ContextMenu','End','!','Meta','Alt','[','c','v','NumLock','Insert','Home','z','AudioVolumeDown',
                 'F2','a','x','$','>',']','*','%','&','s','<','AudioVolumeMute','+','_','`','{','#','F12','@','pauses',
                 'pause_time','end_punctuation','sentence_length','Move_Ratio','Paste_Ratio','Replace_Ratio',
                 'Nonproduction_Ratio','Remove_Ratio','Input_Ratio','revision_events','Revision_Ratio','characters_per_pause',
                'Paste_Time_Ratio','Replace_Time_Ratio','Nonproduction_Time_Ratio','Remove_Time_Ratio','Input_Time_Ratio',
                 'revision_time','Revision_Time_Ratio','action_time_per_pause','action_time_per_pause_time',
                 'characters_per_word']
    
    # Selecting only features that impact model performance
    df = df[keep_cols]
    
    # Adding additional features
    df['characters_per_word_count'] = df['q'] / df['word_count']
    df.loc[df['word_count']==0,'characters_per_word_count'] = df['q']
    
    df['word_count_per_comma'] = df['word_count'] / df[',']
    df.loc[df[',']==0,'word_count_per_comma'] = df['word_count']
    
    df['characters_per_comma'] = df['q'] / df[',']
    df.loc[df[',']==0,'characters_per_comma'] = df['word_count']
    
    df['word_count_per_shift'] = df['word_count'] / df['Shift']
    df.loc[df['Shift']==0,'word_count_per_shift'] = df['word_count']
    
    df['characters_per_shift'] = df['q'] / df['Shift']
    df.loc[df['Shift']==0,'characters_per_shift'] = df['q']
    
    df['characters_per_enter'] = df['q'] / df['Enter']
    df.loc[df['Enter']==0,'characters_per_enter'] = df['q']
    
    df['words_per_enter'] = df['word_count'] / df['Enter']
    df.loc[df['Enter']==0,'words_per_enter'] = df['word_count']
    
    df['word_count_per_hyphen'] = df['word_count'] / df['-']
    df.loc[df['-']==0,'word_count_per_hyphen'] = df['word_count']
    
    df['characters_per_hyphen'] = df['q'] / df['-']
    df.loc[df['-']==0,'characters_per_hyphen'] = df['word_count']
    
    return df

### 3c. Preparing for Final Round of Modeling

In [64]:
# Applying second preprocessing function

final_df = preprocess2(full_df)
final_df

<ipython-input-63-c8ef74ed1849>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['characters_per_word_count'] = df['q'] / df['word_count']
<ipython-input-63-c8ef74ed1849>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_count_per_comma'] = df['word_count'] / df[',']
C:\Users\Hendricks\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,id,score,action_time,action_time_%_total,word_count,total_words,Input_time,Nonproduction_time,Paste_time,Remove/Cut_time,...,characters_per_word,characters_per_word_count,word_count_per_comma,characters_per_comma,word_count_per_shift,characters_per_shift,characters_per_enter,words_per_enter,word_count_per_hyphen,characters_per_hyphen
0,001519c8,3.5,297243,0.165135,255,348.0,243731.0,18506.0,0.0,34130.0,...,4.652299,6.349020,21.250000,134.916667,9.444444,59.962963,404.750000,63.750000,255.0,255.0
1,0022f953,3.5,275391,0.152995,320,369.0,237891.0,13781.0,71.0,23550.0,...,4.037940,4.656250,15.238095,70.952381,3.298969,15.360825,248.333333,53.333333,64.0,298.0
2,0042269b,6.0,421201,0.234001,404,549.0,353718.0,33951.0,0.0,32905.0,...,5.289617,7.188119,17.565217,126.260870,10.358974,74.461538,170.823529,23.764706,404.0,2904.0
3,0059420b,2.0,189596,0.105331,206,244.0,167790.0,3062.0,160.0,18410.0,...,4.254098,5.038835,68.666667,346.000000,3.029412,15.264706,346.000000,68.666667,206.0,206.0
4,0075873a,4.0,313702,0.174279,252,339.0,266515.0,6988.0,0.0,40199.0,...,4.545723,6.115079,10.500000,64.208333,6.461538,39.512821,154.100000,25.200000,252.0,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,3.5,499670,0.277594,273,620.0,426990.0,5203.0,0.0,67253.0,...,4.587097,10.417582,8.531250,88.875000,1.664634,17.341463,406.285714,39.000000,273.0,2844.0
2467,ffbef7e5,4.0,214221,0.119012,438,450.0,203403.0,6583.0,0.0,4118.0,...,4.164444,4.278539,18.250000,78.083333,4.132075,17.679245,156.166667,36.500000,438.0,438.0
2468,ffccd6fd,1.5,231580,0.128656,201,213.0,214677.0,10232.0,0.0,6671.0,...,4.549296,4.820896,100.500000,484.500000,201.000000,969.000000,80.750000,16.750000,201.0,201.0
2469,ffec5b38,5.0,289439,0.160799,413,472.0,263216.0,5624.0,0.0,20599.0,...,5.002119,5.716707,15.296296,87.444444,7.942308,45.403846,393.500000,68.833333,413.0,2361.0


In [65]:
# Defining independent and dependent variables

X_final = final_df.drop(['id','score'],axis=1)
y_final = final_df['score']

In [66]:
# Creating column transformer

ct_final = ColumnTransformer(transformers = [
    ('ss', StandardScaler(), X_final.columns)
])

## 4. Final Modeling

I use my strongest models for one final round of modeling that incorporates the feature selection and feature engineering that I performed after the first round of modeling. After fitting my three strongest models to the data and recording the new results of each individual model, I put all three models in a stacking model to improve generalization on unseen data.

In [67]:
# Creating pipeline

fin_rf_pipe = Pipeline([
    ('ct', ct_final),
    ('FIN_RF',RandomForestRegressor(max_depth=7,
                                   n_estimators=200,
                                   min_samples_leaf=2,
                                   min_samples_split=2,
                                   random_state=42))
])

In [68]:
fin_rf_pipe.fit(X_final,y_final)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Nonproduction_time', 'Paste_time', 'Remove/Cut_time',
       'Replace_time', 'event_count',
       ...
       'characters_per_word', 'characters_per_word_count',
       'word_count_per_comma', 'characters_per_comma', 'word_count_per_shift',
       'characters_per_shift', 'characters_per_enter', 'words_per_enter',
       'word_count_per_hyphen', 'characters_per_hyphen'],
      dtype='object', length=111))])),
                ('FIN_RF',
                 RandomForestRegressor(max_depth=7, min_samples_leaf=2,
                                       n_estimators=200, random_state=42))])

In [69]:
# Creating pipeline

fin_gb_pipe = Pipeline([
    ('ct', ct_final),
    ('FIN_GB', GradientBoostingRegressor(learning_rate=.05,
                                        max_depth=3,
                                        n_estimators=100,
                                        random_state=42))
])

In [70]:
fin_gb_pipe.fit(X_final,y_final)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Nonproduction_time', 'Paste_time', 'Remove/Cut_time',
       'Replace_time', 'event_count',
       ...
       'characters_per_word', 'characters_per_word_count',
       'word_count_per_comma', 'characters_per_comma', 'word_count_per_shift',
       'characters_per_shift', 'characters_per_enter', 'words_per_enter',
       'word_count_per_hyphen', 'characters_per_hyphen'],
      dtype='object', length=111))])),
                ('FIN_GB',
                 GradientBoostingRegressor(learning_rate=0.05,
                                           random_state=42))])

In [71]:
# Creating pipeline

fin_xgb_pipe = Pipeline([
    ('ct',ct_final),
    ('FIN_XGB',XGBRegressor(learning_rate = .05, 
                           max_depth = 3,
                           min_child_weight = 3,
                           n_estimators = 100,
                           random_state=42))
])

In [72]:
fin_xgb_pipe.fit(X_final,y_final)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Nonproduction_time', 'Paste_time', 'Remove/Cut_time',
       'Replace_time', 'event_count',
       ...
       'characters_per_word', 'characters_per_word_count',
       'word_count_per_comma', 'characters_per_comma', 'word_coun...
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.05,
                              max_delta_step=0, max_depth=3, min_child_weight=3,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              random_state=42, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [73]:
results_df = results(fin_rf_pipe, results_df, X_final, y_final)

In [74]:
results_df = results(fin_gb_pipe, results_df, X_final, y_final)

In [75]:
# Reviewing results

results_df = results(fin_xgb_pipe, results_df, X_final, y_final)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622
6,GS_ADA,0.6751,0.5654
7,XGB,0.6732,0.5674
8,GS_XGB,0.6414,0.6074
9,FIN_RF,0.6550,0.5907


After performing the final round of feature selection and feature engineering, all three models perform slightly better than they did beforehand. To improve generalization and predictive performance, I pass all three models through a stack model with linear regression as the final estimator. Then, I adjust my previous results function to work for the stack model.

In [76]:
# Identifying the three best models

estimator_list = [
    ('rf', fin_rf_pipe),
    ('gb', fin_gb_pipe),
    ('xgb', fin_xgb_pipe)
]

In [77]:
# Building the stacking model using a linear regression as the final estimator

stack_model = StackingRegressor(
                    estimators=estimator_list,
                    final_estimator=LinearRegression(),
)

In [78]:
def stack_results(pipe, results_df, X_final, y_final):
    # Defining predictions
    y_pred = pipe.predict(X_final)
    
    # Defining a function to calculate RMSE
    def rmse(y_true, y_pred):
        return -np.sqrt(((y_true - y_pred) ** 2).mean())

    # Making RMSE a scorer using make_scorer
    rmse_scorer = make_scorer(rmse, greater_is_better=False)
    
    # Defining a function to calculate R2
    def r2_scorer(y_true, y_pred):
        return r2_score(y_true, y_pred)

    # Making r2_scorer
    r2_scorer = make_scorer(r2_scorer)
    
    # Using KFold for cross-validation to reduce computational expense
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Calculating RMSE and R2 scores
    cv_rmse = round(np.mean(cross_val_score(pipe, X_final, y_final, cv=kf, scoring=rmse_scorer, n_jobs=-1)), 4)
    cv_r2 = round(np.mean(cross_val_score(pipe, X_final, y_final, cv=kf, scoring=r2_scorer, n_jobs=-1)), 4)
    
    # Naming model for results_df
    model_name = 'stack'
    
    # Appending results to the dataframe
    results_df = results_df.append({'Model': model_name, 'RMSE': cv_rmse, 'R2': cv_r2}, ignore_index=True)
    
    return results_df

In [79]:
stack_model.fit(X_final,y_final)

StackingRegressor(estimators=[('rf',
                               Pipeline(steps=[('ct',
                                                ColumnTransformer(transformers=[('ss',
                                                                                 StandardScaler(),
                                                                                 Index(['action_time', 'action_time_%_total', 'word_count', 'total_words',
       'Input_time', 'Nonproduction_time', 'Paste_time', 'Remove/Cut_time',
       'Replace_time', 'event_count',
       ...
       'characters_per_word', 'characters_per_word_count',
       'word_count_per_comm...
                                                             importance_type='gain',
                                                             interaction_constraints='',
                                                             learning_rate=0.05,
                                                             max_delta_step=0,
                                                             max_depth=3,
                                                             min_child_weight=3,
                                                             missing=nan,
                                                             monotone_constraints='()',
                                                             n_estimators=100,
                                                             n_jobs=0,
                                                             num_parallel_tree=1,
                                                             random_state=42,
                                                             reg_alpha=0,
                                                             reg_lambda=1,
                                                             scale_pos_weight=1,
                                                             subsample=1,
                                                             tree_method='exact',
                                                             validate_parameters=1,
                                                             verbosity=None))]))],
                  final_estimator=LinearRegression())

In [80]:
# Adding results to the final dataframe

results_df = stack_results(stack_model, results_df, X_final, y_final)
results_df

,Model,RMSE,R2
0,Dummy,1.0247,0.0000
1,RF,0.6635,0.5801
2,GS_RF,0.6564,0.5889
3,GB,0.6466,0.6010
4,GS_GB,0.6430,0.6055
5,ADA,0.6775,0.5622
6,GS_ADA,0.6751,0.5654
7,XGB,0.6732,0.5674
8,GS_XGB,0.6414,0.6074
9,FIN_RF,0.6550,0.5907


The stacking model has the best performance in terms of both rmse and r2.

## 5. Results and Conclusions

In this final section, I interpret the results and share conclusions and next steps.

### 5a. Interpretation of Results

My final model had cross-validation scores of .63 rmse and .61 r2. The test data scorer on Kaggle does not include r2, but my final rmse on the public leaderboard was .622.

For the students, educators, and researchers who would be the main stakeholders for this project, it is interesting that the model was able to explain over 60% of the variance in essay scores without any access to the actual text of the essay.  While most analyses of essay quality may focus on features like the quality of the argument or the use of evidence, it is useful to know that factors like word count and character count have predictive importance for the score of the essay as well.

To dive deeper into an interpretation of results, I review the feature importances after the second round of feature selection and feature engineering.

In [88]:
# Finding the feature importances from the random forest pipeline

fin_rf_mod = fin_rf_pipe.named_steps['FIN_RF']
fin_rf_impt = fin_rf_mod.feature_importances_

In [89]:
# Creating a dataframe with feature names and importances

fin_cols = list(X_final.columns)
fin_impt_df = pd.DataFrame({'features': fin_cols, 'rf_impt':fin_rf_impt})

In [91]:
# Repeating the process for gradient boost pipeline

fin_gb_mod = fin_gb_pipe.named_steps['FIN_GB']
fin_gb_impt = fin_gb_mod.feature_importances_
fin_impt_df['gb_impt'] = fin_gb_impt

In [92]:
# Repeating the process for xgb pipeline

fin_xgb_mod = fin_xgb_pipe.named_steps['FIN_XGB']
fin_xgb_impt = fin_xgb_mod.feature_importances_
fin_impt_df['xgb_impt'] = fin_xgb_impt

In [93]:
# Creating features for total and mean importance

fin_impt_df['sum'] = fin_impt_df['rf_impt'] + fin_impt_df['gb_impt'] + fin_impt_df['xgb_impt']
fin_impt_df['avg'] = fin_impt_df['sum'] / 3

In [95]:
fin_impt_df.sort_values(by='avg',ascending=False).head(15)

,features,rf_impt,gb_impt,xgb_impt,sum,avg
2,word_count,0.476844,0.469678,0.212907,1.159429,0.386476
18,q,0.158517,0.186026,0.158179,0.502723,0.167574
26,",",0.047197,0.080619,0.045606,0.173422,0.057807
110,characters_per_hyphen,0.018146,0.021308,0.084685,0.124139,0.041380
10,Input,0.031130,0.027265,0.043931,0.102326,0.034109
101,characters_per_word,0.027831,0.030709,0.011424,0.069965,0.023322
103,word_count_per_comma,0.020939,0.016672,0.014556,0.052166,0.017389
82,sentence_length,0.013575,0.021191,0.012728,0.047494,0.015831
36,-,0.001935,0.012851,0.020881,0.035668,0.011889
9,event_count,0.007139,0.004319,0.020016,0.031475,0.010492


After the second round of feature selection and feature engineering, word_count and character count remain the two features with the highest predictive importance. The number of input events was also in the top 5 of feature importance. For essay writers, educators, and researchers, this suggests that more is usually more.  Writers who produce more words and characters tend to earn higher scores on their essays.

Factors that point towards the complexity of words and sentences-- including features related to the use of commas and hyphens as well as characters per word and sentence length-- also had some of the stronger importances. This suggests that essays that use a writing style that includes longer and more complex words and sentences tend to earn higher scores as well.

The absence of certain features in the top 15 is also of interest. Features that point towards revision-- such as deletions, replacements, or moving text-- do not have the predictive importance of features related to text production. This may relate to the conditions surrounding the task: writers had only 30 minutes to work, so it may have been more beneficial to use that limited time to produce more text than it was to revise or edit. Still, in this context, the 'good writing is rewriting' maxim did not seem to apply.

To synthesize, the feature importances suggest that writing more words and more characters correlated with earning higher scores. Producing longer words and sentences with commas and hyphens did, too. Actions related to revision had weaker correlations to the final score.

### 5b. Next Steps

If I were going to continue working on this project, my main next steps would involve additional feature engineering.

In the first notebook, I would do more work around the activity type to try to identify more specifically what is happening in each event. For instance, looking at how many characters were replaced during a replace event or what specifically was removed during a remove event would give the model better access to the exact changes occurring in the text.

I would also create more features around word and sentence variety by using spaces and end punctuation to approximate the ends of words and sentences and then creating a function to count how many words and characters had been input since the previous space or end punctuation. My current features only show the mean word and sentence length; being able to approximate the length of each specific word and sentence would be useful. It would also allow me to create ratios to show how often the writer used words and sentences of different lengths.

I would also attempt to do some additional hyperparameter tuning during modeling and try using a neural network to see if it could produce stronger results.